In [0]:
import tweepy
import time
import pandas as pd
import json
import time

In [0]:
"""
These keys/tokens require a Twitter developer account. Sign up for a developer 
account at https://developer.twitter.com and create a new app.
""" 

api_key = 'aoGosXnGsr6KM1VHpISICxRPI'
secret_key = 'fEyygkhWQK6aaIHqONLdPhhRs4hvqrgyZu3OuWE6PCo2bUXQus'
access_token = '141198705-IlSTJwpgWm5PrOPbpO35pRJv5OxIAAu8b67Alo9j'
secret_token = 'd4yu7qhV80Wn2gXKDHuLJ3mb1osEYH88ck91mIWbkBC9a'

def authenticate(api_key, secret_key, access_token, secret_token):

  auth = tweepy.OAuthHandler(api_key, secret_key)
  auth.set_access_token(access_token, secret_token)
  api = tweepy.API(auth)
  
  return api


api = authenticate(api_key, secret_key, access_token, secret_token)

In [0]:
# Some api calls will result in a rate limit. This functions handles those if needed 
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(5)

To kickstart scraping of influencer accounts, we need a starting point. Here's a top 10 list, and we'll put the account names into a list:
https://www.brandwatch.com/blog/influential-lifestyle-bloggers/


In [0]:
handles_list = ['StacieinAtlanta', 
                'momsreview4you',
                'dawnchats',
                'GiveawayBandit',
                'daddysweeney',
                'pennywisemama',
                'dodomesticdad',
                'TheTypical_Mom',
                'Crissy',
                'EvrydayShortcut']

def get_initial_influencers(handles):
  influencer_id_list = [api.get_user(handle).id for handle in infl_init_handle_list]
  
  return influencer_id_list
 
  
initial_influ_ids = get_initial_influencers(handles_list)

We need to look for mentioned accounts that *aren't* brands. If an influencer mentioned or 
retweeted an account that doesn't look like a brand, there's a very good chance that that account is also an influencer promoting themselves or promoting their own product through other influencer channels. We'll take those accounts as influencers. 

In [8]:
def scrape_influencers_use_google():
  from googlesearch import search
  from random import randint as ranint
  
  list_urls = []
  
  urls_checked = 0
  urls_appended = 0
  while urls_appended < 100:
    for url in search("site:twitter.com social media influencers", 
                      start=urls_checked, 
                      num=10, 
                      pause=ranint(7, 12)):
      
      if '/lists/' in url:
        list_urls.append(url)
        urls_appended += 1
      
      urls_checked += 1
      print(urls_checked)
      
      time.sleep(ranint(7, 12))
      
  list_urls = [url[8:] for url in list_urls] # Removes leading https://

  print(len(list_urls))
  
scrape_influencers_use_google()
    

HTTPError: ignored

In [0]:
likely_influencer_ids = []

def get_mentioned_accounts(root_account):
  tweets = []
  for i in range(1, 6): # 20 statuses at a time returned, range(1,6) returns 100 statuses 
    tweets.append(api.user_timeline(id=root_account, page=i))
    
  tweets = [json.dumps(status._json) for page_pull in tweets for status in page_pull] 
  tweets = [json.loads(status)["text"] for status in tweets]
  
  mentioned_handles = []
  for status in tweets:
    # Ignore retweets
    if status.startswith('RT'):
      pass
    # Ignore direct tweets at other accounts
    elif status.startswith('@'):
      pass
    else:
      split_status = status.split(' ')
      for word in split_status:
        if word.startswith('@'):
          mentioned_handles.append(word)
  
  return {root_account: mentioned_handles}

In [148]:
mentioned_accounts_dict = {}

for user_id in initial_influ_ids:
  mentioned_accounts_dict[user_id] = get_mentioned_accounts(user_id)
  
print(mentioned_accounts_dict)

{15293352: {15293352: ['@autumnatlanta', '@autumnatlanta']}, 169997085: {169997085: ['@jif…', '@YouTube', '@julieredshoes']}, 178203295: {178203295: ['@benandhollyUS', '@cakenknife', '@RealSimple', '@DianasBananas', '@EasyHomeMeals', '@indelight', '@easyhomemeals', '@naturenates!', '@cakenknife', '@GortonsSeafood!', '@Living_Lou.…', '@dailyburn']}, 325805405: {325805405: ['@NewAirUSA', '@giveawaybandit', '@conservamome', '@sennheiserusa', '@BestBuy']}, 958462227664855041: {958462227664855041: []}, 199551313: {199551313: ['@Tide', '@thepkpway']}, 89551657: {89551657: ['@Ford', '@Ford']}, 911766210: {911766210: ['@twitter.com/thetypical_mom', '@bantambagels', '@Starbucks', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.com/thetypical_mom', '@twitter.